## Imports

In [104]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

## Load Dataset

In [2]:
path_dir = './vehicles/vehicles/'

folder_list = os.listdir(path_dir)
file_list = {}
all_vehicles = []
for index, i in enumerate(folder_list[1:]):
    file_list[i] = os.listdir(path_dir + i)

for key in file_list.keys():
    for value in file_list[key][1:]:
        all_vehicles.append(cv2.imread(path_dir + key +'/'+ value))

In [3]:
len(all_vehicles)

8792

In [4]:
path_dir = './non-vehicles/non-vehicles/'

folder_list = os.listdir(path_dir)
file_list = {}
non_vehicles = []
for index, i in enumerate(folder_list[1:]):
    file_list[i] = os.listdir(path_dir + i)

for key in file_list.keys():
    for value in file_list[key][1:]:
        non_vehicles.append(cv2.imread(path_dir + key +'/'+ value))

In [5]:
len(non_vehicles)

8968

In [6]:
y_vehicles = np.ones(8792).astype(int)
y_non = np.zeros(8968).astype(int)

In [7]:
x_train = np.concatenate([all_vehicles, non_vehicles], axis=0)

y_train = np.concatenate([y_vehicles, y_non], axis=0)

In [8]:
x_train.shape

(17760, 64, 64, 3)

In [9]:
len(y_train)

17760

## Feature Extraction : histogram, hog, resize 함수생성

In [ ]:
# 히스토그램 함수생성

In [53]:
def hist_(x):

    hist_flat0=[]
    for img in x:
        hist = cv2.calcHist(images=[img], channels=[0], mask=None,
                        histSize=[32], ranges=[0, 256]) 
        hist_flat0.append(hist.flatten())

    hist_flat1=[]
    for img in x:
        hist = cv2.calcHist(images=[img], channels=[1], mask=None,
                        histSize=[32], ranges=[0, 256]) 
        hist_flat1.append(hist.flatten())

    hist_flat2=[]
    for img in x:
        hist = cv2.calcHist(images=[img], channels=[2], mask=None,
                        histSize=[32], ranges=[0, 256]) 
        hist_flat2.append(hist.flatten())
        
    hist_flat0 = np.array(hist_flat0)
    hist_flat1 = np.array(hist_flat1)
    hist_flat2 = np.array(hist_flat2)
    hist_flat = np.hstack((hist_flat0, hist_flat1, hist_flat2))
        
    return hist_flat

In [56]:
x_hist = hist_(x_train)

In [57]:
x_hist.shape

(17760, 96)

In [ ]:
# hog_hsv 함수생성

In [59]:
x_hsv.shape

(17760, 64, 64, 3)

In [85]:
def hog_hsv(x):
    hsv = []
    for img in x:
        hsv.append(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))
    
    x_hsv = np.array(hsv)
    
    hog_features = []
    hog_images=[]
    h, s, v = x_hsv[:,:,:,0], x_hsv[:,:,:,1], x_hsv[:,:,:,2]
    
    #h
    hog_features_h = []
    hog_images_h=[]
    for img in h:
        hog_feature, hog_image = hog(img, orientations=9, pixels_per_cell=(8,8),
                                    cells_per_block=(2,2),
                                    block_norm='L2-Hys', transform_sqrt=False,
                                    visualize=True, feature_vector=False)
        hog_features_h.append(hog_feature.flatten())
        hog_images_h.append(hog_image)
    
    #s
    hog_features_s = []
    hog_images_s=[]
    for img in s:
        hog_feature, hog_image = hog(img, orientations=9, pixels_per_cell=(8,8),
                                    cells_per_block=(2,2),
                                    block_norm='L2-Hys', transform_sqrt=False,
                                    visualize=True, feature_vector=False)
        hog_features_s.append(hog_feature.flatten())
        hog_images_s.append(hog_image)
    
    #v
    hog_features_v = []
    hog_images_v =[]
    for img in v:
        hog_feature, hog_image = hog(img, orientations=9, pixels_per_cell=(8,8),
                                    cells_per_block=(2,2),
                                    block_norm='L2-Hys', transform_sqrt=False,
                                    visualize=True, feature_vector=False)
        hog_features_v.append(hog_feature.flatten())
        hog_images_v.append(hog_image)
        
    hog_features = np.hstack((np.array(hog_features_h),np.hstack((np.array(hog_features_s),np.array(hog_features_v)))))
    hog_images = np.hstack((np.array(hog_images_h),np.hstack((np.array(hog_images_s),np.array(hog_images_v)))))
                           
    return hog_features, hog_images

In [86]:
x_hog = hog_hsv(x_train)
x_hog_features = x_hog[0]
x_hog_images = x_hog[1]
x_hog_features.shape

(17760, 5292)

In [ ]:
# resize

In [96]:
def resize(x):
    
    resized_x = []
    for img in x:
        resized = cv2.resize(img,(32,32))
        resized_x.append(resized.flatten())
    resized_flat = np.array(resized_x)
    
    return resized_flat

In [98]:
x_resize = resize(x_train)
x_resize.shape

(17760, 3072)

## 함수변환데이터 병합 및 표준화

In [99]:
x_prepared = np.concatenate([x_hist, x_hog_features, x_resize], axis=1)
x_prepared.shape

(17760, 8460)

In [ ]:
# 표준화

In [116]:
Scaler = StandardScaler().fit(x_prepared)
x_scaled = Scaler.transform(x_prepared)

## Data Preparation

In [101]:
Xtrain, Xval, Ytrain, Yval = train_test_split(x_scaled, y_train, test_size = 0.3, stratify=y_train, random_state = 999)

In [102]:
len(Xtrain), len(Xval)

(12432, 5328)

In [103]:
Xtrain.shape

(12432, 8460)

## Classifier

In [32]:
#1 raw image

#skfold = StratifiedKFold(n_splits = 3, shuffle=True, random_state=999)

# 파라미터
C = [0.01, 0.1, 1, 5, 10, 15, 20, 50]
gamma = [0.001, 0.01, 0.1]

# 파라미터 그리드
params = {'C': C, 'gamma': gamma}

# 교차검증
svc_grid = GridSearchCV(estimator=SVC(probability=True), param_grid=params, cv=3, scoring="accuracy", n_jobs = -1)

# 모델 fitting
svc_grid.fit(Xtrain, Ytrain)

# 결과
print(svc_grid.best_score_)
print(svc_grid.best_params_)

0.6785714285714285
{'C': 5, 'gamma': 0.001}


In [ ]:
pred = svc_grid.best_estimator_.predict(Xval)

In [56]:
print(roc_auc_score(Yval, pred))
print(accuracy_score(Yval, pred))

0.6979303065575757
0.6983858858858859


In [72]:
#2 hog 적용
svc = SVC(probability=True, C = 5, gamma = 0.001, random_state=999)
svc.fit(Xtrain, Ytrain)
pred = svc.predict(Xval)
print(roc_auc_score(Yval, pred))
print(accuracy_score(Yval, pred))

0.968133034206944
0.9682807807807807


In [89]:
#3 hog + hist
svc = SVC(probability=True, C = 0.1,random_state=999)
svc.fit(Xtrain, Ytrain)
pred = svc.predict(Xval)
print(roc_auc_score(Yval, pred))
print(accuracy_score(Yval, pred))

0.5172479150871873
0.5219594594594594


In [106]:
#4 grid search

# hist(3channels) + hog(hsv) + resize(32,32) + standardscaled 

skfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=999)

# 파라미터
C = [0.01, 0.1, 1, 5]

# 파라미터 그리드
params = {'C': C}

# 교차검증
svc_grid = GridSearchCV(estimator=LinearSVC(max_iter = 10000, random_state=999), param_grid=params, cv=skfold, scoring="accuracy", n_jobs = -1)

# 모델 fitting
svc_grid.fit(Xtrain, Ytrain)

# 결과
print(svc_grid.best_score_)
print(svc_grid.best_params_)

#예측
pred = svc_grid.best_estimator_.predict(Xval)
print(roc_auc_score(Yval, pred))
print(accuracy_score(Yval, pred))

0.9904279081473056
{'C': 0.01}
0.9928342131444627
0.9928678678678678


In [105]:
# hist(3channels) + hog(hsv) + resize(32,32) + standardscaled
svc = LinearSVC(max_iter=10000, C = 0.01, random_state=999)
svc.fit(Xtrain, Ytrain)
pred = svc.predict(Xval)
print(roc_auc_score(Yval, pred))
print(accuracy_score(Yval, pred))

0.9926446756160323
0.9926801801801802


## Data Saving to Pickle

In [117]:
orient=9
pixels_per_cell=8
cells_per_block=2
spatial = 32
hist_bins = 32

In [119]:
import pickle
dist_pickle = {}
dist_pickle["svc"] = svc_grid.best_estimator_
dist_pickle["scaler"] = Scaler
dist_pickle["orient"] = orient
dist_pickle["pix_per_cell"] = pixels_per_cell
dist_pickle["cell_per_block"] = cells_per_block
dist_pickle["spatial"] = spatial
dist_pickle["hist_bins"] = hist_bins
pickle.dump(dist_pickle, open("svc_pickle.p", 'wb') )
